In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from predrain import *

## Preprocessing

In [ ]:
Predrain.convert_to_tf_records('../data/SRAD2018/', 1, '../PATH', )

In [14]:
#  def convert_to_tf_records(cls, base_path, idx, PATH, is_test, pred_mode, nt=None,
#                               stop=None):
%run predrain convert_to_tf_records  ../data/SRAD2018/ 1,2 ../data False contiguous --nt 10

Processing subdirectory: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:19<00:00, 256.33it/s]
1it [00:19, 19.56s/it]

KeyboardInterrupt: 

In [18]:
# generate stats for normalization
%run train_prednet.py generate_stats ../data/SRAD2018 1 ../data

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [19:03<00:00,  4.37it/s]


In [18]:
# generate stats for normalization
%run train_prednet.py generate_stats ../data/SRAD2018 1,2 ../data

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [19:03<00:00,  4.37it/s]


In [4]:
predrain = Predrain()

Entry point (after initialization)

In [6]:
# ??predrain.set_flags
??predrain.train

Signature: predrain.train(sz=128, nt=10, bs=12, version=1, num_gpus=1, gpu_start=0)
Docstring: <no docstring>
Source:   
    def train(self, sz=128, nt=10, bs=12, version=1, num_gpus=1, gpu_start=0):
        self.set_flags(sz=sz, nt=nt, bs=bs, version=version, num_gpus=num_gpus, gpu_start=gpu_start)
        
        aug_tfms = self.get_tfms()
        
        model = self.get_model()
        
        trn_tensors, val_tensors = self.get_data()
        
        self.fit()
File:      ~/jupyter/tianchi/pred-rain/yang/predrain.py
Type:      method


## Data

In [10]:
import multiprocessing

# multiprocessing.cpu_count()
print(multiprocessing.cpu_count())

32


In [9]:
??predrain.get_data

Signature: predrain.get_data(base_path=PosixPath('../data/tfrecords'), mode='contiguous', idx_list=[1], val_split=0.1, num_parallel_calls=32, buffer_size=3)
Docstring: <no docstring>
Source:   
    def get_data(self, base_path=Path('../data/tfrecords'), mode='contiguous', idx_list=[1],
                 val_split=0.1, num_parallel_calls=32, buffer_size=3):
        # train_1_contiguous_10
        fn_records = ['train_' + str(idx) + '_' + str(mode) + '_' + str(self.nt) for idx in idx_list]
        fns = [str(Path(base_path)/o) for o in fn_records]
        
        trn_tensors = input_fn(bs, sz, nt, aug_tfms, fns, is_val=False, val_split=val_split,
                                     num_parallel_calls=num_parallel_calls, shuffle=True,
                                     buffer_size=buffer_size)
        val_tensors = input_fn(bs, sz, nt, aug_tfms, fns, is_val=True, val_split=val_split,
                                     num_parallel_calls=num_parallel_calls, shuffle=False,
            

In [11]:
??predrain.input_fn

Signature: predrain.input_fn(bs, sz, nt, aug_tfms, fns, is_val=False, val_split=0.1, stats_sep=',', num_parallel_calls=8, shuffle=False, buffer_size=2, shuffle_buffer_size=24)
Source:   
    def input_fn(self, bs, sz, nt, aug_tfms, fns, is_val=False, val_split=0.1,
                 stats_sep=',', num_parallel_calls=8, 
                 shuffle=False, buffer_size=2, shuffle_buffer_size=24):
        """
        TODO: data interleave
        Create tf.data.Iterator from tfrecord file.

        Receives: 
            fns = a list of tfrecords files **without suffix**
            sz  = the dimension the input images are resized to (int)
            aug_tfms = a list of transforms to apply to input images
        """
        fn_records = [o + '.tfrecords' for o in fns]
        dataset = tf.data.TFRecordDataset(fn_records)

        def parser_train(serialized_example):
            # experimental. TODO: read only needed samples
            shape = (nt, 501, 501, 3)
            context_features

In [7]:
??predrain.get_tfms

Signature: predrain.get_tfms()
Docstring: <no docstring>
Source:   
    def get_tfms(self):
        # data augmentation
        # not implemented yet. see the TODO section in README.md
        # from utils.transforms import *
        aug_tfms = []
        return aug_tfms
File:      ~/jupyter/tianchi/pred-rain/yang/predrain.py
Type:      method


In [8]:
??predrain.get_model

Signature: predrain.get_model(optimizer=<tensorflow.python.keras.optimizers.SGD object at 0x7f07cadd3b38>)
Docstring: <no docstring>
Source:   
    def get_model(self, optimizer=SGD(lr=0.002, momentum=0.9, nesterov=True)):
        # TODO: 
        #   1. add experiments to argument list
        #   2. load models from files
        
        from models.prednet_refactored import PredNetCell, PredNet
        n_channels, im_height, im_width = (1, sz, sz)
        input_shape = (im_height, im_width, n_channels)
        stack_sizes = (n_channels, 48, 96, 192)
        R_stack_sizes = stack_sizes
        A_filt_sizes = (3, 3, 3)
        Ahat_filt_sizes = (3, 3, 3, 3)
        R_filt_sizes = (3, 3, 3, 3)

        layer_loss_weights = np.array([1., 0., 0., 0.])  # weighting for each layer in final loss; 
                                                        # "L_0" model:  [1, 0, 0, 0], "L_all": [1, 0.1, 0.1, 0.1]
        layer_loss_weights = np.expand_dims(layer_loss_weights, 1)
        time_l